In [ ]:
from hyperopt import hp,fmin,tpe,STATUS_OK,Trials
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import time
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve,accuracy_score,log_loss
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve,accuracy_score,log_loss
from sklearn.metrics import precision_recall_curve,average_precision_score
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay,balanced_accuracy_score,accuracy_score

In [ ]:
%%time
df=pd.read_csv("/home/anjali/Desktop/MolTox/statistical_correction/bonferroni/train_test_data_10449_bonferroni.csv")

In [ ]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df=df.fillna(0)

In [ ]:
y=df['Toxicity']
X=df.drop(['Toxicity'],axis=1)
np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler

# Select only the numerical columns
numerical_columns = X.columns[(X.dtypes == 'float64') & (X.columns != 'IPC')]

# Create a DataFrame with only the numerical columns
numerical_X = X[numerical_columns]

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler on the numerical data and transform it for the train set
scaled_numerical_X_train = scaler.fit_transform(numerical_X.loc[X_train.index])

# Replace the scaled numerical data back into the original DataFrame for the train set
X.loc[X_train.index, numerical_columns] = scaled_numerical_X_train

# Transform the numerical data for the test set
scaled_numerical_X_test = scaler.transform(numerical_X.loc[X_test.index])

# Replace the scaled numerical data back into the original DataFrame for the test set
X.loc[X_test.index, numerical_columns] = scaled_numerical_X_test



In [ ]:
df=pd.concat([X,y],axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train,test=train_test_split(df,test_size=0.20,random_state=42)

## Hyperparameter Tuning

In [ ]:
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials,rand
%time

space = {
    'criterion': hp.choice('criterion', ['entropy', 'gini']),
    'max_depth': hp.randint('max_depth', 6,13),  # Expanded range for max_depth
    'min_samples_leaf': hp.randint('min_samples_leaf', 1, 200),  # Expanded range for min_samples_leaf
    'min_samples_split': hp.randint('min_samples_split', 2, 56),  # Expanded range for min_samples_split
    'n_estimators': hp.choice('n_estimators', [50,75, 100, 125,100, 200, 300, 500])
}


def objective(space,random_seed=6):
    model = RandomForestClassifier(criterion = space['criterion'], 
                                   max_depth = space['max_depth'],
                                 min_samples_leaf = space['min_samples_leaf'],
                                 min_samples_split = space['min_samples_split'],
                                 n_estimators = space['n_estimators'],
                                class_weight='balanced', random_state=200)
    
    accuracy = cross_val_score(model, X_train, y_train, cv = 5).mean()
    

    # We aim to maximize accuracy, therefore we return it as a negative value
    return {'loss': -accuracy, 'status': STATUS_OK }
    
trials = Trials()
best = fmin(fn= objective,
            space= space,
            algo= tpe.suggest,
            max_evals = 10,
            trials= trials,
           return_argmin=False,
        
           rstate = np.random.default_rng(42))
best

In [ ]:
best